In [3]:
import pandas as pd
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset.drop('User ID', axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [9]:
dep=dataset[['Purchased']]
dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep,test_size=0.30,random_state=0)


In [11]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
X_train=scx.fit_transform(X_train)
X_test=scx.transform(X_test)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid={
    'penalty':['l1','l2','elasticnet',None],
    'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga']
}
grid=GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,Y_train)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
55 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1172, in fit
    solver = _check_solver(self.solver, self.penalty, self.dua

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [13]:
Y_pred=grid.predict(X_test)
Y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1], dtype=int64)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
print(cm)

[[74  5]
 [ 8 33]]


In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,Y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [21]:
from sklearn.metrics import f1_score,accuracy_score
f1_weighted=f1_score(Y_test,Y_pred, average='weighted')
print(f1_weighted)

accuracy=accuracy_score(Y_test,Y_pred)
print("Accuracy of the best model {} : {:.2f}".format(grid.best_params_, accuracy))

0.8906190214115365
Accuracy of the best model {'penalty': 'l1', 'solver': 'liblinear'} : 0.89


In [18]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(Y_test, grid.predict_proba(X_test)[:,1])
print("roc_auc score for best parameter {}: {:.2f}".format(grid.best_params_,roc_score ))

roc_auc score for best parameter {'penalty': 'l1', 'solver': 'liblinear'}: 0.95


In [19]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005281,0.004518,0.000000,0.000000,l1,lbfgs,"{'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,0.011153,0.004071,0.031067,0.003603,l1,liblinear,"{'penalty': 'l1', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
2,0.001198,0.001469,0.000000,0.000000,l1,newton-cg,"{'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
3,0.001918,0.000951,0.000000,0.000000,l1,newton-cholesky,"{'penalty': 'l1', 'solver': 'newton-cholesky'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
4,0.001477,0.001236,0.000000,0.000000,l1,sag,"{'penalty': 'l1', 'solver': 'sag'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
5,0.013799,0.008667,0.021134,0.007277,l1,saga,"{'penalty': 'l1', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
6,0.027190,0.009288,0.019157,0.003996,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
7,0.008931,0.002337,0.018195,0.002367,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
8,0.012186,0.001988,0.016086,0.001287,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
9,0.101712,0.072838,0.016021,0.001458,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
